<h1>Using the yfinance Library</h1>

<h3>Imports</h3>

In [151]:
import yfinance as yf
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score, mean_absolute_error, r2_score, mean_squared_error

import pandas as pd
import numpy as np
import plotly.offline as plyo
import cufflinks as cf

<h3>Grabbing Tickers</h3>

In [152]:
all_tickers = "WFC MSFT INTC AMZN"
selected_stocks = yf.Tickers(all_tickers)
tickers = all_tickers.split(" ")

<h3>Getting the History for Each Ticker</h3>

In [153]:
selected_history = {}

for index in range(len(tickers)):
    selected_history[tickers[index]] = selected_stocks.tickers[index].history(period="1y")

<h3>Viewing Data</h3>

In [154]:
selected_history[tickers[0]].info()
selected_history[tickers[0]].head(10)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 253 entries, 2019-10-24 to 2020-10-23
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          253 non-null    float64
 1   High          253 non-null    float64
 2   Low           253 non-null    float64
 3   Close         253 non-null    float64
 4   Volume        253 non-null    int64  
 5   Dividends     253 non-null    float64
 6   Stock Splits  253 non-null    int64  
dtypes: float64(5), int64(2)
memory usage: 15.8 KB


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-10-24,48.461860,48.978430,48.461860,48.882767,16695400,0.0,0
2019-10-25,48.863638,49.485432,48.787107,49.332375,17433300,0.0,0
2019-10-28,49.686315,49.791543,49.332371,49.408901,23914800,0.0,0
2019-10-29,49.265411,50.088096,49.265411,49.906338,21063000,0.0,0
2019-10-30,49.628926,50.011568,49.380205,49.772415,16155900,0.0,0
2019-10-31,49.523695,49.772412,48.930598,49.389771,18814700,0.0,0
2019-11-01,49.868079,50.021136,49.657624,49.915909,16359100,0.0,0
2019-11-04,50.298551,50.509003,49.973304,50.432476,17559900,0.0,0
2019-11-05,50.451613,51.178635,50.451613,50.987312,25965300,0.0,0


<h3>Analysis Methods</h3>

In [155]:
def createLinearRegression(X, y):
    lr = LinearRegression()
    lr.fit(X, y)
    return lr

def testLinearRegression(model, X_train, X_test, y_train, y_test):
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)
    
    preds = model.predict(X_test)
    score = explained_variance_score(y_test, preds)
    mae = mean_absolute_error(y_test, preds)
    rmse = math.sqrt(mean_squared_error(y_test, preds))
    r2 = r2_score(y_test, preds)
    
    print("Train Score: {}\nTest Score: {}"
          .format(train_score, test_score))
    print("Score Details = {:.5f} | MAE = {:.3f} | RMSE = {:.3f} | R2 = {:.5f}"
          .format(score, mae, rmse, r2))
    print("\n")
    

def conductLinearRegressionAnalysis(data_set, features, target):
    X = data_set[features]
    y = data_set[target]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
    
    model= createLinearRegression(X_train, y_train)
    testLinearRegression(model, X_train, X_test, y_train, y_test)
    
    return model

<h3>Models for Selected Tickers</h3>

In [156]:
features = ["Open", "High", "Low", "Volume"]
target = "Close"
models = {}

for key, history in selected_history.items():
    print("For: {}".format(key))
    models[key] = conductLinearRegressionAnalysis(history, features, target)

For: WFC
Train Score: 0.9988896371135679
Test Score: 0.9996600550484013
Score Details = 0.99967 | MAE = 0.162 | RMSE = 0.213 | R2 = 0.99966


For: MSFT
Train Score: 0.9969802209733848
Test Score: 0.9923111494352979
Score Details = 0.99256 | MAE = 1.450 | RMSE = 2.380 | R2 = 0.99231


For: INTC
Train Score: 0.9913000248687956
Test Score: 0.9868842356035019
Score Details = 0.98689 | MAE = 0.392 | RMSE = 0.628 | R2 = 0.98688


For: AMZN
Train Score: 0.9987158036482552
Test Score: 0.9985468363126084
Score Details = 0.99856 | MAE = 17.184 | RMSE = 23.114 | R2 = 0.99855




<h3>Getting Data for Last 5 Days</h3>

In [157]:
selected_5d_history = {}

for index in range(len(tickers)):
    selected_5d_history[tickers[index]] = selected_stocks.tickers[index].history(period="1mo")

<h3>Prediction Methods</h3>

In [158]:
def createPredictions(model, data_set, features):
    return model.predict(data_set[features])

<h3>Predicting the Last 5 Days</h3>

In [159]:
for key, history in selected_5d_history.items():
    history["Prediction"] = createPredictions(models[key], history, features)

<h3>Plotting Methods</h3>

In [160]:
def plotResults(data_set, plot_values):
    plyo.iplot(data_set[plot_values].iplot(asFigure=True))

<h3>Plotting Actual vs Prediction</h3>

In [161]:
plot_values = ["Close", "Prediction"]

for key, history in selected_5d_history.items():
    print("For {}:".format(key))
    plotResults(history, plot_values)

For WFC:


For MSFT:


For INTC:


For AMZN:
